In [4]:
from torch.utils.data import DataLoader
from moses import CharVocab, StringDataset
from moses.char_rnn import CharRNN, CharRNNTrainer
from moses.char_rnn import config as CharRNNConfig
import moses
import torch
import rdkit.Chem as chem

train = moses.get_dataset('train')

In [5]:
config = CharRNNConfig.get_config()
config.log_file = None
config.n_workers = 1
config.n_batch = 256
config.model_save = 'crnn.pth'
config.save_frequency = 10
trainer = CharRNNTrainer(config)
crnn = CharRNN(CharVocab.from_data(train), config)
crnn.load_state_dict(torch.load('crnn._030.pt'))

<All keys matched successfully>

In [6]:
torch.cuda.set_device(0)

In [7]:
torch.cuda.is_initialized()

True

In [8]:
crnn.cuda()

CharRNN(
  (embedding_layer): Embedding(30, 30, padding_idx=28)
  (lstm_layer): LSTM(30, 768, num_layers=3, batch_first=True, dropout=0.2)
  (linear_layer): Linear(in_features=768, out_features=30, bias=True)
)

In [9]:
crnn.device

device(type='cuda', index=0)

In [10]:
trainer.fit(crnn, train)

/home/darius/anaconda3/envs/moses/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


CharRNN(
  (embedding_layer): Embedding(30, 30, padding_idx=28)
  (lstm_layer): LSTM(30, 768, num_layers=3, batch_first=True, dropout=0.2)
  (linear_layer): Linear(in_features=768, out_features=30, bias=True)
)

In [11]:
torch.save(crnn, "crnn_full_model.pth")

In [12]:
crnn

CharRNN(
  (embedding_layer): Embedding(30, 30, padding_idx=28)
  (lstm_layer): LSTM(30, 768, num_layers=3, batch_first=True, dropout=0.2)
  (linear_layer): Linear(in_features=768, out_features=30, bias=True)
)

In [17]:
import pandas as pd
from io import StringIO
import sys
chem.WrapLogs()
sio = sys.stderr = StringIO()


total_per_n = 2_000_000
save_every = 1_000

with open('crnn_smiles.csv', 'w') as fh:
    fh.write(f"smiles,valid\n")


for n in range(1):
    for i in range(total_per_n // save_every):
        vals = []
        smiles = crnn.sample(save_every)
        for smile in smiles:
            mol = chem.MolFromSmiles(smile)
            if mol is None:
                sio = sys.stderr = StringIO()
            vals.append((smile, mol is not None))
        print(f"Saving for n={n}, {i + 1}/{total_per_n // save_every}")
        with open('crnn_smiles.csv', 'a') as fh:
            for s, valid in vals:
                fh.write(f"{s},{valid}\n")

Saving for n=0, 1/1000
Saving for n=0, 2/1000
Saving for n=0, 3/1000
Saving for n=0, 4/1000
Saving for n=0, 5/1000
Saving for n=0, 6/1000
Saving for n=0, 7/1000
Saving for n=0, 8/1000
Saving for n=0, 9/1000
Saving for n=0, 10/1000
Saving for n=0, 11/1000
Saving for n=0, 12/1000
Saving for n=0, 13/1000
Saving for n=0, 14/1000
Saving for n=0, 15/1000
Saving for n=0, 16/1000
Saving for n=0, 17/1000
Saving for n=0, 18/1000
Saving for n=0, 19/1000
Saving for n=0, 20/1000
Saving for n=0, 21/1000
Saving for n=0, 22/1000
Saving for n=0, 23/1000
Saving for n=0, 24/1000
Saving for n=0, 25/1000
Saving for n=0, 26/1000
Saving for n=0, 27/1000
Saving for n=0, 28/1000
Saving for n=0, 29/1000
Saving for n=0, 30/1000
Saving for n=0, 31/1000
Saving for n=0, 32/1000
Saving for n=0, 33/1000
Saving for n=0, 34/1000


KeyboardInterrupt: 